<a href="https://colab.research.google.com/github/AndreMartins21/Eleicoes_2022/blob/main/infos_eleicoes_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Classe Mãe (EleicaoBase)**

In [ ]:
import requests
import json
import pandas as pd


class EleicaoBase:
    def __init__(self, uf: str = "mg"):
        self.uf = uf.lower()
        self.url = self.get_url()

    def get_data(self) -> pd.DataFrame:
        r = requests.get(self.url)

        json_data = json.loads(r.content)
        info_candidatos = json_data["cand"]

        df = pd.DataFrame(info_candidatos)
        # Tratando dados
        df["seq"] = df["seq"].astype(int)
        df["votos_int"] = df["vap"].astype(int)
            
        df["vap"] = df["vap"].apply(
            lambda qtd_voto: f"{int(qtd_voto):,}"
        )

        # Ordenando e filtrando colunas
        df = df.sort_values(["seq"]).reset_index(drop=True)
        df = df[
            ["n", "nm", "vap", "pvap", "cc", "votos_int"]
        ]
        
        df.rename(
            columns={
              "cc": "Partido",
              "n": "Numero", 
              "nm": "Nome", 
              "vap": "Votos Apurados",
              "pvap": "% de votos"
            }, inplace=True
        )
        return df

    def search(self, name: str) -> pd.DataFrame:
        df = self.get_data()
        name = name.upper()
        search = df[
            df["Nome"].str.contains(name)
        ]

        return search
    
    def get_rank_by_partido(self) -> pd.DataFrame:
        df = self.get_data()
        values = df[["Partido", "Nome", "votos_int"]].groupby(["Partido"]).agg(
            {"votos_int": "sum"}
        ).sort_values(
            by=["votos_int"], ascending = False
        )

        values["votos_int"] = values["votos_int"].apply(
            lambda qtd_voto: f"{int(qtd_voto):,}"
        )
        
        return values  

## **Presidente**

In [ ]:
class Presidente(EleicaoBase):
    def __init__(self, **kwargs):
        super().__init__()
    
    @staticmethod
    def get_url() -> str:
        return "https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/br/br-c0001-e000544-r.json"

In [ ]:
Presidente().get_data()

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
0,13,LULA,"56,047,441","48,09",PT - Federação Brasil da Esperança - FE BRASIL...,56047441
1,22,JAIR BOLSONARO,"50,678,241","43,49",PL - PP / REPUBLICANOS / PL,50678241
2,15,SIMONE TEBET,"4,883,444","4,19",MDB - MDB / Federação PSDB Cidadania (PSDB/CID...,4883444
3,12,CIRO GOMES,"3,556,324","3,05",PDT,3556324
4,44,SORAYA THRONICKE,"592,988","0,51",UNIÃO,592988
5,30,FELIPE D&apos;AVILA,"557,068","0,48",NOVO,557068
6,14,PADRE KELMON,"79,653","0,07",PTB,79653
7,80,LÉO PÉRICLES,"53,242","0,05",UP,53242
8,21,SOFIA MANZANO,"45,206","0,04",PCB,45206
9,16,VERA,"25,396","0,02",PSTU,25396


## **Governador**

In [ ]:
class Governador(EleicaoBase):
    def __init__(self, uf: str = "mg"):
        super().__init__(uf)
    
    def get_url(self) -> str:
        return f"https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{self.uf}/{self.uf}-c0003-e000546-r.json"        

In [ ]:
Governador(
    uf = "mg"
).get_data()

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
0,30,ZEMA,"6,027,152","56,27",NOVO - PP / PODE / SOLIDARIEDADE / PATRIOTA / ...,6027152
1,55,KALIL,"3,745,537","34,97",PSD - Federação Brasil da Esperança - FE BRASI...,3745537
2,22,CARLOS VIANA,"777,260","7,26",PL - PL / REPUBLICANOS / PRTB,777260
3,45,MARCUS PESTANA,"59,559","0,56",PSDB - Federação PSDB Cidadania (PSDB/CIDADANI...,59559
4,50,LORENE FIGUEIREDO,"44,655","0,42",PSOL - Federação PSOL REDE (PSOL/REDE),44655
5,35,CABO TRISTAO,"15,612","0,15",PMB,15612
6,80,INDIRA XAVIER,"15,473","0,14",UP,15473
7,21,RENATA REGINA,"12,400","0,12",PCB,12400
8,16,VANESSA PORTUGAL,"11,900","0,11",PSTU,11900
9,29,LOURDES FRANCISCO,"1,996","0,02",PCO,1996


## **Deputado Federal**

In [ ]:
class DepFederal(EleicaoBase):
    def __init__(self, uf: str = "mg"):
        super().__init__(uf)
    
    def get_url(self) -> str:
        return f"https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{self.uf}/{self.uf}-c0006-e000546-r.json"

In [ ]:
DepFederal(
    uf = "mg"
).get_data().head()

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
0,2222,NIKOLAS FERREIRA,"1,481,696","13,40",PL,1481696
1,7040,ANDRÉ JANONES,"235,783","2,13",AVANTE,235783
2,1212,DUDA SALABERT,"207,315","1,88",PDT,207315
3,1312,REGINALDO LOPES,"194,176","1,76",PT - Federação Brasil da Esperança - FE BRASIL...,194176
4,1313,ROGÉRIO CORREIA,"183,795","1,66",PT - Federação Brasil da Esperança - FE BRASIL...,183795


In [ ]:
dep_fed_sp = DepFederal(
    uf = "sp"
)

dep_fed_sp.get_rank_by_partido().head(3)

,votos_int
Partido,
PL,"5,148,621"
PT - Federação Brasil da Esperança - FE BRASIL (PT/PC do B/PV),"2,538,249"
PSOL - Federação PSOL REDE (PSOL/REDE),"1,961,261"


In [ ]:
dep_fed_sp.get_data().head(8)

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
0,5010,GUILHERME BOULOS,"999,439","4,22",PSOL - Federação PSOL REDE (PSOL/REDE),999439
1,2210,CARLA ZAMBELLI,"945,194","3,99",PL,945194
2,2222,EDUARDO BOLSONARO,"740,707","3,13",PL,740707
3,2250,RICARDO SALLES,"640,204","2,70",PL,640204
4,1111,DELEGADO BRUNO LIMA,"460,571","1,94",PP,460571
5,4040,TABATA AMARAL,"337,340","1,42",PSB,337340
6,1000,CELSO RUSSOMANNO,"304,998","1,29",REPUBLICANOS,304998
7,4433,KIM KATAGUIRI,"294,988","1,25",UNIÃO,294988


## **Deputado Estadual**

In [ ]:
class DepEstadual(EleicaoBase):
    def __init__(self, uf: str = "mg"):
        super().__init__(uf)
    
    def get_url(self) -> str:
        return f"https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{self.uf}/{self.uf}-c0007-e000546-r.json"

In [ ]:
dep_est = DepEstadual(
    uf = "mg"
)

dep_est.get_rank_by_partido().head()

,votos_int
Partido,
PT - Federação Brasil da Esperança - FE BRASIL (PT/PC do B/PV),"1,387,992"
PL,"1,225,475"
PSD,"953,037"
PP,"759,379"
REPUBLICANOS,"461,103"


In [ ]:
dep_est.get_data().head()

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
0,22222,BRUNO ENGLER,"629,882","5,79",PL,629882
1,13123,BEATRIZ CERQUEIRA,"244,814","2,25",PT - Federação Brasil da Esperança - FE BRASIL...,244814
2,55555,CASSIO SOARES,"131,071","1,20",PSD,131071
3,22123,ANTONIO CARLOS ARANTES,"118,795","1,09",PL,118795
4,10800,MAURO TRAMONTE,"109,724","1,01",REPUBLICANOS,109724


In [ ]:
dep_est.search("Douglas Melo")

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
41,55800,DOUGLAS MELO,"61,462","0,56",PSD,61462


## **Senador**

In [ ]:
class Senador(EleicaoBase):
    def __init__(self, uf: str = "mg"):
        super().__init__(uf)
    
    def get_url(self) -> str:
        return f"https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/{self.uf}/{self.uf}-c0005-e000546-r.json"

In [ ]:
Senador(
    uf = "mg"
).get_data()

,Numero,Nome,Votos Apurados,% de votos,Partido,votos_int
0,200,CLEITINHO,"4,204,844","41,65",PSC,4204844
1,555,ALEXANDRE SILVEIRA,"3,600,350","35,66",PSD - PSD / PSB / Federação Brasil da Esperanç...,3600350
2,111,MARCELO ARO,"1,987,555","19,69",PP - NOVO / MDB / SOLIDARIEDADE / PATRIOTA / A...,1987555
3,500,SARA AZEVEDO,"112,098","1,11",PSOL - Federação PSOL REDE (PSOL/REDE),112098
4,123,BRUNO MIRANDA,"104,141","1,03",PDT - Federação PSDB Cidadania (PSDB/CIDADANIA...,104141
5,142,PASTOR ALTAMIRO ALVES,"46,500","0,46",PTB,46500
6,161,DIRLENE MARQUES,"36,920","0,37",PSTU,36920
7,290,NAOMI DE ALMEIDA,"3,192","0,03",PCO,3192
